In [ ]:
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from scipy.optimize import minimize
from scipy.linalg import solve_triangular
from scipy.special import kv

sys.path.append('/Users/xiangshi/Documents/GitHub/normix/')

from normix.gig import GIG
from normix.gh import GH
from normix.func import logkv, kvratio, logkvp

%matplotlib inline

In [4]:
s1 = 0.3127893130513858
s2 = 6.180567062222285
s3 = 1.4941762114948172

def llh(param):
    lam = param[0]
    chi = param[1]
    psi = param[2]
    eta = np.sqrt(chi*psi)
    delta = np.sqrt(chi/psi)
    l = 0.5*(chi*s1+psi*s2)-(lam-1)*s3+lam*np.log(delta)+logkv(lam, eta)
    return l

def grad(param):
    lam = param[0]
    chi = param[1]
    psi = param[2]
    eta = np.sqrt(chi*psi)
    delta = np.sqrt(chi/psi)
    g = np.array([-s3+np.log(delta)+(logkv(lam+1e-10, eta)-logkv(lam-1e-10, eta))/2e-10,
                  0.5*(s1+lam/chi+logkvp(lam, eta)/delta),
                  0.5*(s2-lam/psi+logkvp(lam, eta)*delta)])
    return g

x0 = np.array([0.1, 1., 1.])
bounds = [(-20., 20.), (0., None), (0., None)]
res = minimize(fun=llh, x0=x0, jac=grad, bounds=bounds)

In [ ]:
res

## EM algorithm: first iteration

In [ ]:
# small sample
x = np.array([[ 1.74953772,  1.37746568, -1.54835839],
       [ 3.76765938,  1.54792901, -0.36846954],
       [ 5.70719392,  0.82978642, -2.07754969],
       [ 1.95051383,  0.62586649, -0.66932423],
       [ 7.93513039,  0.54582224,  0.90509363],
       [11.64517463, -0.26257948,  7.62055921],
       [ 0.47822655,  2.261273  , -0.68519925],
       [14.36924369,  2.9331593 , -0.39317326],
       [ 3.06647501,  0.94545944,  0.87562929],
       [ 2.36628983,  1.99396013, -0.60114535]])

In [ ]:
# mu0 = np.array([ 0.62270148,  1.42936867, -1.12930602])
# gamma0 = np.array([ 2.0634707 , -0.06592856,  0.63264504])
# sigma0 = np.array([[ 2.14201689, -0.2556575 ,  0.39176087],
#        [-0.2556575 ,  0.40719298, -0.29542022],
#        [ 0.39176087, -0.29542022,  1.4751854 ]])
# lam0 = 0.09194905476463931
# chi0 = 1.7076264456008048
# psi0 = 0.7226005679030028

# mu0 = np.array([ 1.74953769,  1.37746568, -1.54835838])
# gamma0 = np.array([11.35000959, -0.31185785,  5.92142542])
# sigma0 = np.array([[16.98033929, -2.96961102, -7.81379197],
#        [-2.96961096,  4.12189925, -1.24987587],
#        [-7.81379203, -1.24987584,  5.5121773 ]])
# lam0 = 0.37757030708097666
# chi0 = 1.4118979442341222e-13
# psi0 = 4.1489465280628846

In [ ]:
mu0 = np.zeros(3)
sigma0 = np.eye(3)
gamma0 = np.zeros(3)
lam0 = 1.0
chi0 = 1.0
psi0 = 1.0
diff = 1e-5

gh_fit = GH(mu0, gamma0, sigma0, lam0, chi0, psi0)
suff_stats = [0]*6
suff_stats[3] = np.mean(x, axis=0)
llh_last = 0


for i in range(10):
    llh, lam, delta, eta = gh_fit.llh(x)

    # E-step
    a = kvratio(lam-1, lam, eta)/delta
    b = kvratio(lam+1, lam, eta)*delta
    c = (kvratio(lam+diff, lam, eta)-kvratio(lam-diff, lam, eta))/(2*diff)+np.log(delta)

    suff_stats[0] = np.mean(a)
    suff_stats[1] = np.mean(b)
    suff_stats[2] = np.mean(c)
    suff_stats[4] = np.mean(x.T*a, axis=1)
    suff_stats[5] = np.dot((x.T*a)/x.shape[0], x)

    gh_fit.mu = (suff_stats[3]-suff_stats[1]*suff_stats[4])/(1-suff_stats[0]*suff_stats[1])
    gh_fit.gamma = (suff_stats[4]-suff_stats[0]*suff_stats[3])/(1-suff_stats[0]*suff_stats[1])
    gh_fit.sigma = -np.outer(suff_stats[4], gh_fit.mu)
    gh_fit.sigma = gh_fit.sigma + gh_fit.sigma.T + suff_stats[5] \
        + suff_stats[0]*np.outer(gh_fit.mu, gh_fit.mu) \
        - suff_stats[1]*np.outer(gh_fit.gamma, gh_fit.gamma)

    gig = GIG(gh_fit.lam, gh_fit.chi, gh_fit.psi)
    res = gig.suffstats2param(suff_stats[0], suff_stats[1], suff_stats[2])
    
    gh_fit.lam = gig.lam
    gh_fit.chi = gig.chi
    gh_fit.psi = gig.psi
    
    gh_fit.regulate()
    
    display(i)
    display(np.mean(llh))
#     display(suff_stats[:3])
    display(gh_fit.mu, gh_fit.gamma, gh_fit.sigma, gh_fit.lam, gh_fit.chi, gh_fit.psi)

In [ ]:
(-2.7165243590771455+2.7228886528801968)/-2.7228886528801968

In [ ]:
s = np.array([[18.31639362, -3.2031186 , -8.43000923],
       [-3.2031186 ,  4.4454594 , -1.34764241],
       [-8.43000923, -1.34764241,  5.94339906]])

In [ ]:
np.linalg.det(s)

In [ ]:
s1, s2, s3 = suff_stats[:3]

def llh(param):
    lam = param[0]
    chi = param[1]
    psi = param[2]
    eta = np.array([np.sqrt(chi*psi)])
    delta = np.sqrt(chi/psi)
    l = 0.5*(chi*s1+psi*s2)-(lam-1)*s3+lam*np.log(delta)+logkv(lam, eta)
    return l[0]

def grad(param):
    lam = param[0]
    chi = param[1]
    psi = param[2]
    eta = np.array([np.sqrt(chi*psi)])
    delta = np.sqrt(chi/psi)
    g = np.array([(-s3+np.log(delta)+(logkv(lam+1e-10, eta)[0]-logkv(lam-1e-10, eta)[0])/2e-10),
                  0.5*(s1+lam/chi+logkvp(lam, eta)[0]/delta),
                  0.5*(s2-lam/psi+logkvp(lam, eta)[0]*delta)])
    return g

x0 = np.array([gh_fit.lam, gh_fit.chi, gh_fit.psi])
bounds = [(-20., 20.), (1e-20, None), (1e-20, None)]
res = minimize(fun=llh, x0=x0, jac=grad, bounds=bounds, options={'disp':True})

In [ ]:
res

In [ ]:
llh([-20, 1e-30, 1e-30])

In [ ]:
llh([20, 1e-30, 1e-30])

In [ ]:
llh([-20, 1e30, 1e-30])

In [ ]:
llh([-20, 1e-30, 1e30])

## EM algorithm

In [5]:
# actual parameters
mu = np.array([0.65634476,  1.51842634, -0.63371085])
sigma= np.array([[0.35646818, 0.10212985, 0.05799233],
       [0.10212985, 0.98229971, 0.15375874],
       [0.05799233, 0.15375874, 1.41077451]])
gamma = np.array([ 2.05705176, -0.04197259,  0.34341511])
lam=0.9
chi=0.3
psi=1.0

# small sample
x = np.array([[ 1.74953772,  1.37746568, -1.54835839],
       [ 3.76765938,  1.54792901, -0.36846954],
       [ 5.70719392,  0.82978642, -2.07754969],
       [ 1.95051383,  0.62586649, -0.66932423],
       [ 7.93513039,  0.54582224,  0.90509363],
       [11.64517463, -0.26257948,  7.62055921],
       [ 0.47822655,  2.261273  , -0.68519925],
       [14.36924369,  2.9331593 , -0.39317326],
       [ 3.06647501,  0.94545944,  0.87562929],
       [ 2.36628983,  1.99396013, -0.60114535]])

In [6]:
gh = GH(mu, gamma, sigma, lam, chi, psi)
gh.regulate()

In [38]:
x = gh.rvs(50000)

In [39]:
mu0 = np.zeros(3)
sigma0 = np.eye(3)
gamma0 = np.zeros(3)
lam0 = 1.0
chi0 = 1.0
psi0 = 1.0
diff = 1e-5

gh_fit = GH(mu0, gamma0, sigma0, lam0, chi0, psi0)
gh_fit.fit_em(x, eps=1e-4)

iter=1, llh=-3.21912, change=3.58238
iter=2, llh=-3.18398, change=0.03514
iter=3, llh=-3.16503, change=0.01895
iter=4, llh=-3.15240, change=0.01263
iter=5, llh=-3.14370, change=0.00870
iter=6, llh=-3.13766, change=0.00604
iter=7, llh=-3.13349, change=0.00417
iter=8, llh=-3.13062, change=0.00287
iter=9, llh=-3.12864, change=0.00198
iter=10, llh=-3.12726, change=0.00138
iter=11, llh=-3.12627, change=0.00099
iter=12, llh=-3.12554, change=0.00073
iter=13, llh=-3.12497, change=0.00057
iter=14, llh=-3.12452, change=0.00045
iter=15, llh=-3.12414, change=0.00037
iter=16, llh=-3.12383, change=0.00032
iter=17, llh=-3.12355, change=0.00028
success


True

In [40]:
np.mean(gh.llh(x)[0])

-3.120738929966929

In [41]:
gh_fit.mu, gh.mu

(array([ 0.82412092,  1.50717916, -0.62292325]),
 array([ 0.65634476,  1.51842634, -0.63371085]))

In [42]:
gh_fit.gamma, gh.gamma

(array([ 2.08335942, -0.0332831 ,  0.3626529 ]),
 array([ 2.64691579, -0.05400832,  0.44189013]))

In [43]:
gh.lam, gh_fit.lam

(0.9, 0.9610170261801337)

In [44]:
gh.chi, gh_fit.chi

(0.23314513108753068, 0.13713815910912858)

In [45]:
gh.psi, gh_fit.psi

(1.2867521556234844, 1.0339679875769607)

In [46]:
gh.sigma, gh_fit.sigma

(array([[0.4586862 , 0.1314158 , 0.07462176],
        [0.1314158 , 1.26397627, 0.19784939],
        [0.07462176, 0.19784939, 1.81531714]]),
 array([[0.65989222, 0.1055972 , 0.10244527],
        [0.1055972 , 1.04789762, 0.17394077],
        [0.10244527, 0.17394077, 1.50981347]]))